# OpenMASTER - quickstart

-------------------------

In [ ]:
import pyomo.environ as pyo
import openMASTER

In [ ]:
m = openMASTER.make_model()
m

Be aware the openMASTER_Data.xlsm file has to be downloaded using git-lfs or the following link:

https://github.com/IIT-EnergySystemModels/openMASTER/raw/main/data/input/openMASTER_Data.xlsm?download=

In [ ]:
%%time
data = openMASTER.load_dataportal_from_excel('../data/input/openMASTER_Data.xlsm')

In [ ]:
%%time
#openMASTER.excel_to_csv('data/input/openMASTER_Data.xlsm')
data = openMASTER.load_dataportal_from_csv()

In [ ]:
%%time
instance = m.create_instance(data)

In [ ]:
%%time
solver = pyo.SolverFactory('gurobi')
#solver.options['Method'] = 2

solver_results = solver.solve(instance, keepfiles=False, tee=True)

### **Results**

In [ ]:
import openMASTER

Extract information on variables through the model output

In [ ]:
path        = "../data/input/openMASTER_Data.xlsm"
output_path = "../data/tmp/output"
sheetname   = "Output"

d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

Variable information to excel

Loading variable information from .csv

In [ ]:
d_vars = openMASTER.import_results_from_csv(output_path)